In [1]:
"""
Created on Tue Feb 19 09:46:13 2019

@author: AYu5
"""
import sys
import pandas as pd
import numpy as np
import string
import re
from decimal import Decimal
import datetime as dt



In [41]:
# Change following 3 variables into current positions and date

input_position = (r'C:\Users\ayu5\Desktop\0409DataParsing\raw data\Fixed')
output_position = (r'C:\Users\ayu5\Desktop\0409DataParsing\output')
date =  '04092019'




In [66]:

# Don't change any the codes below unless necessary and certain

input_position = input_position.replace('\\','/')
output_position = output_position.replace('\\','/')

function_name_list = ['Actor','Casting','Consultant','Director','Financier',
                 'Rights','Generic_Functions','Producer','Researcher','Writer']

def existed_change(existed_fix):
    df = existed_fix.copy()
    if 'Index1' in df.columns:
        df = df.drop(columns=['Index1'])
    return df

def new_change(new_fix):
    df = new_fix.copy()
    df = df.rename(index=str, 
                   columns={'Darts_Division':'Right_DARTS_DIVISION','COMPENSATION_ID':'Right_COMPENSATION_ID',
                           'COMPENSATION_AMOUNT':'Right_COMPENSATION_AMOUNT','COMPENSATION_DESC':'Right_COMPENSATION_DESC',
                           'COMPENSATION_TYPE':'Right_COMPENSATION_TYPE','DEAL_ID':'Right_DEAL_ID',
                           'FUNCTION':'Right_FUNCTION','PROJECT_ID':'Right_PROJECT_ID'})
    return df

#First step, read files
dict_combined = {}
for func in function_name_list:
    if func == 'Rights':
        print(func)
        existed_file = pd.ExcelFile(input_position + '/' + "Delta_Compensation_Rights_Fixed_In&Out_Existed.xlsx")
        new_file = pd.ExcelFile(input_position + '/' + "Delta_Compensation_Rights_Fixed_In&Out_New.xlsx")
        existed_df = pd.read_excel(existed_file,"Sheet1")
        new_df = pd.read_excel(new_file,"Sheet1")
    else:
        existed_file = pd.ExcelFile(input_position + '/' + "Delta_Compensation_" + func + "_Fixed_Existed.xlsx")
        new_file = pd.ExcelFile(input_position + '/' + "Delta_Compensation_" + func + "_Fixed_New.xlsx")
        existed_df = pd.read_excel(existed_file,"Sheet1")
        new_df = pd.read_excel(new_file,"Sheet1")
    
    combined = pd.concat([existed_change(existed_df),new_change(new_df)],sort=False,ignore_index=True)
    dict_combined[func] = combined

Rights


### Variable Preparation

Convert "Right_COMPENSATION_ID" into string.

Add one temporary column named "split_desc", which is used to specify split content for each line, which will be droped at the end.

Date time type convert. For "Compensation.Start_Date","Rights Start Date", "Compensation Commitment Date","Start_Date","Expiry_Date","Purchase_Date","Reversion_Date",
"Date Paid".

In [67]:
date_columns = ["Compensation.Start_Date","Rights Start Date", "Compensation Commitment Date","Start_Date","Expiry_Date","Purchase_Date","Reversion_Date", "Date Paid"]

In [68]:
for df in dict_combined.values():
    
    
    
    df['split_desc'] = df["Right_COMPENSATION_DESC"]
    
    
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col],errors='coerce')
    
    #df["Right_COMPENSATION_ID"] = df["Right_COMPENSATION_ID"].fillna(-1)
    #df["Right_COMPENSATION_ID"] = df["Right_COMPENSATION_ID"].astype('int64')
    #df["Right_COMPENSATION_ID"] = df["Right_COMPENSATION_ID"].astype('str')
    #df["Right_COMPENSATION_ID"] = df["Right_COMPENSATION_ID"].replace("-1", np.nan)
    
    #df = df.astype('str')
    

In [69]:
dict_combined["Rights"].head(10)

,Compensation.Applicable_ind,Compensation.Duration_#,Compensation.Duration_Freq,Compensation.Payment_Type,Compensation.%_Total,Compensation.Start_Qualifier,Compensation.Payment_Start_Condition,Index,DARTS_DIVISION,COMPENSATION_ID,...,Date Paid,Right_DARTS_DIVISION,Right_COMPENSATION_ID,Right_COMPENSATION_AMOUNT,Right_COMPENSATION_DESC,Right_COMPENSATION_TYPE,Right_DEAL_ID,Right_FUNCTION,Right_PROJECT_ID,split_desc
0,NaN,12.0,Month,NaN,NaN,NaN,NaN,Original,SCGM,321.0,...,NaT,SCGM,321.0,25000.00,SIGNED - For 12 month option to purchase right...,Guaranteed,321.0,Rights,U31012,SIGNED - For 12 month option to purchase right...
1,NaN,12.0,Month,NaN,NaN,NaN,NaN,Original,SCGM,322.0,...,NaT,SCGM,322.0,25000.00,"For 12 month extension, non-app.",Optional,321.0,Rights,U31012,"For 12 month extension, non-app."
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Original,SCGM,323.0,...,NaT,SCGM,323.0,350000.00,(less initial option fee and writing fixed com...,Purchase Price,321.0,Rights,U31012,(less initial option fee and writing fixed com...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Original,SCGM,335.0,...,NaT,SCGM,335.0,119726.00,Purchase Price payment,Guaranteed,328.0,Rights,U29305,Purchase Price payment
4,NaN,6.0,Month,NaN,NaN,NaN,NaN,Original,SCGM,361.0,...,NaT,SCGM,361.0,15000.00,"For 6 mos. Initial option to purchase rights, ...",Guaranteed,341.0,Rights Team,X69203,"For 6 mos. Initial option to purchase rights, ..."
5,NaN,6.0,Month,NaN,NaN,NaN,NaN,Original,SCGM,362.0,...,NaT,SCGM,362.0,15000.00,"For 6 mos. second option to purchase rights, n...",Guaranteed,341.0,Rights Team,X69203,"For 6 mos. second option to purchase rights, n..."
6,NaN,2.0,Year,NaN,NaN,NaN,NaN,Original,SCGM,363.0,...,NaT,SCGM,363.0,185000.00,Purchase Price payment. The 2 year reversion ...,Guaranteed,341.0,Rights Team,X69203,Purchase Price payment. The 2 year reversion ...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Original,SCGM,366.0,...,NaT,SCGM,366.0,1109129.64,Payment of consideration to Warner Bros.,Guaranteed,342.0,Rights,X48134,Payment of consideration to Warner Bros.
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Original,SCGM,367.0,...,NaT,SCGM,367.0,119726.00,Payment of Purchase Price ($150K less writing ...,Guaranteed,343.0,Rights,X67636,Payment of Purchase Price ($150K less writing ...
9,NaN,24.0,Month,NaN,NaN,NaN,NaN,Original,SCGM,381.0,...,NaT,SCGM,381.0,100000.00,"For 24 mos. option to purchase rights, app.",Guaranteed,361.0,Rights,U31006,"For 24 mos. option to purchase rights, app."


### Copy Combined raw data

In [60]:
dict_combined["Actor"].head()

,Compensation.Applicable_ind,Compensation.Compensation_Type,Compensation.Fee_Type,Compensation.Start_Date,Compensation.Duration_#,Compensation.Duration_Freq,Compensation.Rate,Compensation.Rate_Freq,Compensation.Payment_Type,Compensation.%_Total,...,PROJECT_ID,Right_DARTS_DIVISION,Right_COMPENSATION_ID,Right_COMPENSATION_AMOUNT,Right_COMPENSATION_DESC,Right_COMPENSATION_TYPE,Right_DEAL_ID,Right_FUNCTION,Right_PROJECT_ID,split_desc
0,NaN,Guaranteed,Acting Fee,NaN,NaN,NaN,10000.0,Day,NaN,NaN,...,KG0307,SPA,1431,50000.0,"for the role of ""Simon"", $10,000/day with a 5-...",Guaranteed,755,Talent,KG0307,"for the role of ""Simon"", $10,000/day with a 5-..."
1,NaN,Guaranteed,Acting Fee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,KG0307,SPA,1433,5000.0,10/17/2006: Agreed to pay 1/2 day salary on t...,Guaranteed,755,Talent,KG0307,10/17/2006: Agreed to pay 1/2 day salary on t...
2,NaN,Guaranteed,Acting Fee,NaN,NaN,NaN,10000.0,Session,NaN,NaN,...,KG0307,SPA,1484,30000.0,"in the role of ""Eunice"" (Bride of Frankenstein...",Guaranteed,790,Talent,KG0307,"in the role of ""Eunice"" (Bride of Frankenstein..."
3,NaN,Guaranteed,Acting Fee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,KG0307,SPA,1485,100000.0,DBO bonuses tied to the initial theatrical rel...,Guaranteed,790,Talent,KG0307,DBO bonuses tied to the initial theatrical rel...
4,NaN,Guaranteed,Acting Fee,NaN,NaN,NaN,10000.0,Session,NaN,NaN,...,KG0307,SPA,1547,80000.0,"in the role of ""Dracula"" -- 5 recording sessio...",Guaranteed,810,Talent,KG0307,"in the role of ""Dracula"" -- 5 recording sessio..."


In [61]:
dict_combined_copy = dict_combined.copy()

In [62]:
dict_combined = dict_combined_copy.copy()

### General Function Preparation

In [9]:
# 1.line spliter
def line_spliter(orig_df,index,num):
    df = orig_df.copy()
    row = df.loc[index]
    print(num - 1)
    for j in range(num - 1):
        df = df.append(row)
    return df

In [10]:
# Transfer money string format from "$1.2K" into "1200"
def money_transfer(money_str):
    string  = money_str
    num = Decimal(re.sub(r'[^\d.]', '', string))
    if 'k' in string or 'K' in string:
        num = num * 1000
    if 'm' in string or 'M' in string:
        num = num * 1000000
    return num

In [11]:
money_transfer("1.21M")

Decimal('1210000.00')

### Data Parsing

### Iterate not the index, but Right_COMPENSATION_ID !!!

In the following steps, each step will input df and output df.

1. First step, copy and paste "Right_COMPENSATION_TYPE".

In [12]:
def comp_type_paster(df, comp_id):
    row = df[df["Right_COMPENSATION_ID"] == comp_id]
    ind = row.index[0]
    df.at[ind, "Compensation.Compensation_Type"] = df.loc[ind, 'Right_COMPENSATION_TYPE']
    return df

2. second step, "Orig deal"

In [13]:
# Remove string behind "Orig deal"
# Not for "New", just for "Updated" at now.
# Not used now.
def orig_deal_paster(df, comp_id):
    row = df[df["Right_COMPENSATION_ID"] == comp_id]
    ind = row.index[0]
    desc = row['split_desc'].values[0]
    # To see if there is "orig deal"
    sub_strs = re.findall(r"(?i)(.*)(Origi?n?a?l? deal.*)", desc)
    if len(sub_strs) == 1:
        df.at[ind, 'split_desc'] = sub_strs[0][0]
    return df

3. third step, "overage".

In [14]:
def overage_spliter(df, comp_id):
    # For now, only 1 row
    row = df[df["Right_COMPENSATION_ID"] == comp_id]
    index = row.index[0]
    desc = row['split_desc'].values[0]
    # To see if there is "orig deal"
    sub_strs = re.findall(r"(?i)(.*)(overage.{4,})", desc)
    #split line
    if len(sub_strs) == 1:
        num = 2
        df = line_spliter(df, index, num)
        df.at[index,'split_desc'] = sub_strs[0]
    return df.reset_index(drop=True)

4. fourth step, "Payment_Type"

In [15]:
def Payment_Type_Regex(str_test):    
    
    # mode1:(25/25/25/25)
    sub_strs = re.findall(r"(?i)25\%(.*)25\%(.*)25\%(.*)25\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"25/25/25/25 Split",['0.25','0.25','0.25','0.25'],sub_strs)
    
    # mode2:(50/50)
    sub_strs = re.findall(r"(?i)50\%(.*)50\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"Equal Installments",['0.5','0.5'],sub_strs)
    
    sub_strs = re.findall(r"(?i)1\/2(.*)1\/2(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"Equal Installments",['0.5','0.5'],sub_strs)
    
    # mode3:(20/60/10/10)
    sub_strs = re.findall(r"(?i)20\%(.*)60\%(.*)10\%(.*)10\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"20/60/10/10 Split",['0.2','0.6','0.1','0.1'],sub_strs)

    # mode4:(20/60/20)
    sub_strs = re.findall(r"(?i)20\%(.*)60\%(.*)20\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"20/60/20 Split",['0.2','0.6','0.2'],sub_strs)
    
    # mode5:(80/10/10)
    sub_strs = re.findall(r"(?i)80\%(.*)10\%(.*)10\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"80/10/10 Split",['0.8','0.1','0.1'],sub_strs)  

    # mode6:(15/60/12.5/12.5)
    sub_strs = re.findall(r"(?i)15\%(.*)60\%(.*)12.5\%(.*)12.5\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"15/60/12.5/12.5 Split",['0.15','0.6','0.125','0.125'],sub_strs)  
    
    # mode7:(60/10/10/10/10)
    sub_strs = re.findall(r"(?i)60\%(.*)10\%(.*)10\%(.*)10\%(.*)10\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"60/10/10/10/10 Split",['0.6','0.1','0.1','0.1','0.1'],sub_strs) 
    
    
    # mode8:(1/3 1/3 1/3)
    sub_strs = re.findall(r"(?i)1\/3(.*)1\/3(.*)1\/3(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"1/3 1/3 1/3 split",['1/3','1/3','1/3'],sub_strs) 
    
    # no mode caught
    return (False,"",[],[])

In [16]:
def Payment_Type_spliter(df, comp_id):
    # For now, there may be more than 1 row
    # do it for every row
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index

    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            
            row = df.loc[ind]
            
            desc = row["split_desc"]
            flag, type_name, percen_total, sub_strs = Payment_Type_Regex(desc)
            
            if flag:

                df = line_spliter(df, ind, len(percen_total))
                df.at[ind, "Compensation.Payment_Type"] = type_name#np.repeat(type_name,len(percen_total))
                df.at[ind, "Compensation.%_Total"] = percen_total
                df.at[ind, "split_desc"] = sub_strs[0]
        
    return df.reset_index(drop=True)

Actor:

1.Over	Scheduled Principal Photography: pybl over sched, payable over schedule, over princ photog, sched of photo comm,
over sched period of prin photo, prin photo, over prin photo,




Director:

2.On Completion of Delivery: on compl del

3.On Completion of Dub/Score: on compl dub/score

4.On Election to Proceed/Abandon: on election to proceed/abandon, elects proceed/ abandon, upon election to proceed or abandonment

5.Over Pre-Production: over 8 wks pre-prod, over 8 wks preprod, over preproduction,


6.Later of: later of

7.Over Production:over production



Producer:

8.Earlier of:earlier of

9.On Commencement of Services: on commencement of screenwriter's services, on commencement of opt. services,  on commencement of writer's services, comm of Teitler/Weber/Field's svcs,

10.On Delivery of Answer Print: answer print

11.On Satisfaction of Conditions Precedent: on satisfaction of conditions precedent, satis. of cond. prec., satisfaction of conditions precedent, satisfn of cond precedent, 



Rights:

12.On Execution of Agreement: on execution of long form agreement, upon execution of the agreement, on execution, upon exec,
on exec



5.Fifth step, other columns to all functions.

In [17]:
def start_quali_condi_regex(str_test):
    
    print(str_test)
    
    # mode1:"Over Scheduled Principal Photography"
    
    # pybl over sched, payable over schedule, over sched period of prin photo
    sub_strs = re.findall(r"(?i)over\sschedu?l?e?", str_test)
    if len(sub_strs) > 0:
        return (True,"Over","Scheduled Principal Photography")
    
    # over princ photog, prin photo, over prin photo
    sub_strs = re.findall(r"(?i)princ?i?p?a?l?\sphoto", str_test)
    if len(sub_strs) > 0:
        return (True,"Over","Scheduled Principal Photography")
    
    # sched of photo comm
    sub_strs = re.findall(r"(?i)schedu?l?e?\sof\sphoto", str_test)
    if len(sub_strs) > 0:
        return (True,"Over","Scheduled Principal Photography")    
    
    
    # model2:"Over Pre-Production"
    # "over 8 wks pre-prod, over 8 wks preprod, over preproduction"
    sub_strs = re.findall(r"(?i)over\s.{,12}pre\-?produ?c?t?i?o?n?", str_test)
    if len(sub_strs) > 0:
        return (True,"Over","Pre-Production")
    
    
    
    # model3: "Over Production"
    # 'over production'
    sub_strs = re.findall(r"(?i)over\s.{,12}produ?c?t?i?o?n?", str_test)
    if len(sub_strs) > 0:
        return (True,"Over","Production")    
    
    
    
    # model4:"On Completion of Delivery"
    # "on compl del"
    sub_strs = re.findall(r"(?i)on\scomple?t?i?o?n?e?\sdeli?v?e?r?y?", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Completion of Delivery") 
    
    
    # model5:"On Completion of Dub/Score"
    # "on compl dub/score"
    sub_strs = re.findall(r"(?i)on\scomple?t?i?o?n?e?\sdub\/score", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Completion of Dub/Score")    
    
    
    # model6:"On Election to Proceed/Abandon"
    # "on election to proceed/abandon, elects proceed/ abandon, upon election to proceed or abandonment"
    sub_strs = re.findall(r"(?i)proceed\W{,2}o?r?\W{,2}abandon", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Election to Proceed/Abandon")    
    
    
    # model7:"On Commencement of Services"
    # "on commencement of screenwriter's services, on commencement of opt. services,
    # on commencement of writer's services, comm of Teitler/Weber/Field's svcs"
    sub_strs = re.findall(r"(?i)comme?n?c?e?m?e?n?t? of .{,15} se?r?vi?ce?s", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Commencement of Services")  
    
    
    # model8:"On Delivery of Answer Print"
    # "answer print"
    sub_strs = re.findall(r"(?i)answer print", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Delivery of Answer Print")      
    
    
    # model 9:"On Satisfaction of Conditions Precedent"
    # "on satisfaction of conditions precedent,satis. of cond. prec.,
    # satisfaction of conditions precedent, satisfn of cond precedent,"
    sub_strs = re.findall(r"(?i)satisf?a?c?t?i?o?n?\.? of condi?t?i?o?n?s?\.? prece?d?e?n?t?\.?", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Delivery of Answer Print")    
    
    
    # model10:"On Execution of Agreement"
    # "on execution of long form agreement, upon execution of the agreement, on execution, upon exec, on exec"
    sub_strs = re.findall(r"(?i)on execu?t?i?o?n?", str_test)
    if len(sub_strs) > 0:
        return (True,"On","Execution of Agreement")     
    
    # model11:"Later of"
    # "later of"
    sub_strs = re.findall(r"(?i)later of", str_test)
    if len(sub_strs) > 0:
        return (True,"Later of","")
    
    # model12:"Earlier of"
    # "earlier of"
    sub_strs = re.findall(r"(?i)earlier of", str_test)
    if len(sub_strs) > 0:
        return (True,"Earlier of","")    
    
    
    
    # no mode caught
    return (False,"","")
    
    
    
    
    

In [18]:
# For "Compensation.Start_Qualifier" and 
# "Compensation.Payment_Start_Condition" or "Compensation.Service_Start_Condition" (Director)
def start_quali_condi_paster(df, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            row = df.loc[ind]
            desc = row["split_desc"]
            print(desc)
            flag, start_quali, start_condi = start_quali_condi_regex(desc)
            
            if flag:
                df.at[ind, "Compensation.Start_Qualifier"] = start_quali
                if 'Compensation.Payment_Start_Condition' in df.columns:
                    df.at[ind, 'Compensation.Payment_Start_Condition'] = start_condi
                elif "Compensation.Service_Start_Condition" in df.columns:
                    df.at[ind,"Compensation.Service_Start_Condition"] = start_condi
    
    return df


def start_quali_condi_regex_modes(str_test):
    

    re1 = r'(?i)over\sschedu?l?e?'
    re2 = '(?i)princ?i?p?a?l?\sphoto'
    re3 = '(?i)schedu?l?e?\sof\sphoto'
    re4 = '(?i)over\s.{,12}pre\-?produ?c?t?i?o?n?'
    re5 = r'(?i)over\sschedu?l?e?'
    re6 = '(?i)princ?i?p?a?l?\sphoto'
    re7 = '(?i)over\s.{,12}produ?c?t?i?o?n?'
    re8 = '(?i)on\scomple?t?i?o?n?e?\sdeli?v?e?r?y?'    
    re9 = '(?i)on\scomple?t?i?o?n?e?\sdub\/score'        
    re10 = '(?i)proceed\W{,2}o?r?\W{,2}abandon'        
    re11 = '(?i)comme?n?c?e?m?e?n?t? of .{,15} se?r?vi?ce?s'        
    re12 = '(?i)answer print'    
    re13 = '(?i)satisf?a?c?t?i?o?n?\.? of condi?t?i?o?n?s?\.? prece?d?e?n?t?\.?'        
    re14 = '(?i)on execu?t?i?o?n?'      
    re15 = '(?i)later of'   
    re16 = '(?i)earlier of'   

    
    generic_re = re.compile("(%s|%s|%s|%s%s|%s|%s|%s%s|%s|%s|%s%s|%s|%s|%s)" % (re1, re2, re3, re4, re5, re6, re7, re8, re9,
                                              re10, re11, re12, re13, re14, re15, re16)).findall(str_test)

    
    return len(generic_re)


### For Compensation.%_Amount

Combine %_Amount and .Payment_Start_Condition together.

In [19]:
# for Compensation.%_Amount
def per_amount_spliter(df, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            row = df.loc[ind]
            desc = row["split_desc"]
            
            result = re.findall(r"(?i)\$\W?([1-9]\d{0,2}[\,\.]?\d{,3}K?M?)",desc)
            
            #if len(result) == 1:
                
            #    df.at[ind,"Compensation.%_Amount"] = money_transfer(result[0])
            
            if len(result) > 1:
                pos = [0]
                moneys = []
                for m in re.finditer(r"(?i)\$\W?([1-9]\d{0,2}[\,\.]?\d{,3}K?M?)",desc):
                    pos.append(m.start())
                    pos.append(m.end())
                    moneys.append(money_transfer(m.groups()[0]))
                    
                pos.append(len(desc))
                i = 0
                flags = 0
                split_desc_list = []
                while i+1 <= len(pos):
                    start_point = pos[i]
                    print(start_point)
                    end_point = pos[i+1]
                    print(end_point)
                    print(desc[start_point:end_point])
                    flag, start_quali, start_condi = start_quali_condi_regex(desc[start_point:end_point])
                    if flag:
                        split_desc_list.append(desc[start_point:end_point])
                        flags = flags + 1
                    i = i + 2

                
            
                
                if len(moneys) == flags:
                    print("cut_amount")
                    print(split_desc_list)
                    df = line_spliter(df, ind, flags)
                    df.at[ind,'Compensation.%_Amount'] = moneys
                    df.at[ind,"split_desc"] = split_desc_list
            else:
                return df.reset_index(drop=True)
    
    return df.reset_index(drop=True)

In [20]:
# For 'Compensation.Duration_#' & 'Compensation.Duration_Freq'.
def dura_num_freq_paster(df, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    
    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            
            row = df.loc[ind]
            
            desc = row["split_desc"]
            capture = 'session|week|wk|month|mo|year|yr|day'
            result = re.findall(r"(?i)\s(?=(\d{1,2}\.?\d?)\W("+capture+r"))\b",desc)
            if len(result) > 0:
                df.at[ind, 'Compensation.Duration_#'] = result[0][0].replace(",","")
                df.at[ind, 'Compensation.Duration_Freq'] = result[0][1].replace("day","Day")\
                .replace("session","Session").replace("wk","Week").replace("week","Week")\
                .replace("mo","Month").replace("month","Month").replace("Monthnth","Month")\
                .replace("yr","Year").replace("year","Year")
            else:
                continue; 
    
    return df

### also remove caught "$" + number, to help "%_Amount" catching

In [21]:
# For 'Compensation.Rate' & 'Compensation.Rate_Freq'.
def rate_num_freq_paster(df, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    
    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            
            row = df.loc[ind]
            
            desc = row["split_desc"]
            capture = 'session|week|wk|month|mo|year|yr|day'
            result = re.findall(r"(?i)\$\W?([1-9]\d{0,2}[\,\.]?\d{,3}K?)\W?(per|\/)\W?("+capture+r")",desc)
            
            
            # remove caught "$" + number, to help "%_Amount" catching
            for m in re.finditer(r"(?i)\$\W?([1-9]\d{0,2}[\,\.]?\d{,3}K?)\W?(per|\/)\W?("+capture+r")",desc):
                desc = desc[:m.start()]+desc[m.end():]
            
            df.at[ind, 'split_desc.Rate'] = desc
            
            
            
            if len(result) > 0:
                
                value = result[0][0]
                
                num = money_transfer(value)
                
                df.at[ind, 'Compensation.Rate'] = num
                df.at[ind, 'Compensation.Rate_Freq'] = result[0][2].replace("day","Day")\
                .replace("session","Session").replace("wk","Week").replace("week","Week")\
                .replace("mo","Month").replace("month","Month").replace("Monthnth","Month")\
                .replace("yr","Year").replace("year","Year")
            else:
                continue; 
                
    
    
    return df

In [40]:
# For 'Compensation.Start_Date'
# from/commencing/comm \b XX/XX/XX
# Not for two date time
def start_date_paster(df, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    
    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            row = df.loc[ind]
            desc = row["split_desc"]
            result = re.findall(r"(?i)(START DATE|comm|commencing|TERM)\W{,3}(\d{1,2}\/\d{1,2}\/\d{2,4})",desc)
            if len(result) > 0:
                df.at[ind, 'Compensation.Start_Date'] = result[0][1]
            else:
                continue; 
    
    return df

6.Sixth step, other non-split-line columns related each specific functions.

### Not very perfect for fee_type, don't know if I should do it with line-spliting when encoutering multiple fee types.

In [23]:
def fee_type_regex(func_name,desc):
    if func_name == "Actor":
        # Acting fee or Overage fee
        sub_strs = re.findall(r"(?i)overage", desc)
        if len(sub_strs) > 0:
            return "Overage Fee"
        else:
            return "Acting Fee"

            
    if func_name == "Casting":
        # Casting fee or Overage fee
        sub_strs = re.findall(r"(?i)overage", desc)
        if len(sub_strs) > 0:
            return "Overage Fee"
        else:
            return "Casting Fee"


    if func_name == "Consultant":
        # Consulting Fee or Overage fee
        sub_strs = re.findall(r"(?i)overage", desc)
        if len(sub_strs) > 0:
            return "Overage Fee"
        else:
            return "Consulting Fee"
    
            
    if func_name == "Director":
        # Directing Fee or Development Fee
        sub_strs = re.findall(r"(?i)development (fee|service)", desc)
        if len(sub_strs) > 0:
            return "Development Fee"
        else:
            return "Directing Fee"
    
            
    if func_name == "Financier":
        return np.nan
            
            
    if func_name == "GenericFunction":
        return np.nan
    
    if func_name == "Producer":
        # Producing Fee or Executive Producer Fee or Development Fee or Co-Producer Fee
        if len(re.findall(r"(?i)development (fee|service)", desc)) > 0:
            return "Development Fee"
        elif len(re.findall(r"(?i)Executive Prod", desc)) > 0 or len(re.findall(r"(?i)EP fee", desc)) > 0:
            return "Executive Producer Fee"
        elif len(re.findall(r"(?i)co\W?prod", desc)) > 0:
            return "Co-Producer Fee"            
        else:
            return "Producing Fee"
        
      
                
    if func_name == "Researcher":
        return np.nan
            
    if func_name == "Writer":
        return "Writing Fee"
      
    if func_name == "Rights":
        return np.nan


In [24]:
def fee_type_paster(df, func_name, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
        
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"]
        
        # func_name: Actor,Casting,Consultant,Director,Financier,
        # GenericFunction,Producer,Researcher,Writer,Rights
        df.at[ind,"Compensation.Fee_Type"] = fee_type_regex(func_name,desc)          
            
    return df
    
    

Compensation.Service_Type only for Producer.

In [25]:
def service_type_regex(desc):
    # Producer Services or Executive Producer Services or Co-Producer Services
    if len(re.findall(r"(?i)Executive Prod", desc)) > 0 or len(re.findall(r"(?i)EP fee", desc)) > 0:
        return "Executive Producer Services"
    elif len(re.findall(r"(?i)co\W?prod", desc)) > 0:
        return "Co-Producer Services"            
    else:
        return "Producer Services"    
    
    

In [26]:
def service_type_paster(df, func_name, comp_id):
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"]    
        df.at[ind,"Compensation.Service_Type"] = service_type_regex(desc)
    return df 

7.Seventh step, unique columns (for Rights and Writer).

Functions for Rights.

In [27]:
# for Compensatipon.Total_Guaranteed_Commitment
def total_guarant_paster(df, comp_id):
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index

    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        
        if row["Right_COMPENSATION_TYPE"] == "Guaranteed":
            df.at[ind,"Compensatipon.Total_Guaranteed_Commitment"] = row["Right_COMPENSATION_AMOUNT"]
        
    return df 

In [28]:
# for Compensation.Check_#
def check_number_paster(df, comp_id):
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"] 
        sub_strs = re.findall(r"(?i)ch?e?c?ks?\W{,3}(\d{3,})", desc)
        
        if len(sub_strs) > 0:
            df.at[ind,"Compensation.Check_#"] = sub_strs[0]
    return df     

In [29]:
# for Start_Date and Expiry_Date
# 1. XX/XX/XX thru/- XX/XX/XX
# 2. EXPIRES 3/29/10
def start_expiry_date_paster(df, comp_id):
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"] 
        
        #1.
        sub_strs = re.findall(r"(?i)(\d{1,2}\/\d{1,2}\/\d{1,2}) (\-|thru) (\d{1,2}\/\d{1,2}\/\d{1,2})", desc)
        
        if len(sub_strs) > 0:
            df.at[ind,"Start_Date"] = pd.to_datetime(sub_strs[0][0])
            df.at[ind,"Expiry_Date"] = pd.to_datetime(sub_strs[0][2])
            
        #2.
        sub_strs = re.findall(r"(?i)expire\w\W{,3}(\d{1,2}\/\d{1,2}\/\d{1,2})", desc)
        
        if len(sub_strs) > 0:
            df.at[ind,"Expiry_Date"] = pd.to_datetime(sub_strs[0])
            
    return df

In [30]:
# for Date Paid
# eg."**Paid via wire on 3/30/17.**"
def date_paid_paster(df, comp_id):
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"] 
        
        sub_strs = re.findall(r"(?i)\*\*.*(\d{1,2}\/\d{1,2}\/\d{1,2}).*\*\*", desc)
        
        if len(sub_strs) > 0:
            df.at[ind,"Date Paid"] = pd.to_datetime(sub_strs[0])
    
    
    return df

### Functions for Writer.

The key task about Writer Parsing is how to split Compensation_Amount---Writing_Step_Amount---%_Amount.

4 types:
1. Only Writing_Step's name, not amount. Just parse the Writing_Step, copy and paste the Right_Compensation_Amount into Writing_Step_Amount.

2. Both Writing_Step's name and amount(money), no condition. Dividing by Writing_Step, then pasring corresponding amount.

3. Both condition and amount(money), no Writing_Step's name. Dividing by Writing_Step, then pasring corresponding amount.

4. All Writing_Step's name, condition and amount(money). Dividing firstly by Writing_Step, then by condition, finally parsing amount.

Sometimes there are things like "1/2" "1/4", etc.

## Change the following replaces, Capital!! Also, don't include ","

In [31]:
def writer_payment_type_regex(str_test):
    
    # mode1:(25/25/25/25)
    sub_strs = re.findall(r"(?i)25\%(.*)25\%(.*)25\%(.*)25\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"25/25/25/25 Split",['0.25','0.25','0.25','0.25'],sub_strs)

    # mode2:(50/50)
    sub_strs = re.findall(r"(?i)50\%(.*)50\%(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"Equal Installments",['0.5','0.5'],sub_strs)
    
    sub_strs = re.findall(r"(?i)1\/2(.*)1\/2(.*)", str_test)
    if len(sub_strs) > 0:
        return (True,"Equal Installments",['0.5','0.5'],sub_strs)
    
    return (False,"",[],[])

In [32]:
def Writer_Payment_Type_spliter(df, comp_id):
    # For now, there may be more than 1 row
    # do it for every row
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index

    if len(indexs) > 0:
        for ind in indexs:
            print(comp_id)
            
            row = df.loc[ind]
            
            desc = row["split_desc"]
            flag, type_name, percen_total, sub_strs = writer_payment_type_regex(desc)
            
            if flag:
                
                total_amount = df.loc[ind,"Right_COMPENSATION_AMOUNT"]

                df = line_spliter(df, ind, len(percen_total))
                df.at[ind, "Compensation.Payment_Type"] = type_name#np.repeat(type_name,len(percen_total))
                df.at[ind, "Compensation.%_Total"] = percen_total
                df.at[ind, "Compensation.%_Amount"] = np.repeat(total_amount * float(percen_total[0]),len(percen_total))
                df.at[ind, "split_desc"] = sub_strs[0]
        
    return df.reset_index(drop=True)

In [33]:
def writing_step_regex(str_test):
    
    catching_list = []
    
    order = '1st|first|2nd|second|3rd|third|4th|fourth|5th|fifth|6th|sixth'
    
    step_names = 'draft|polish|rewrite|revision|outline|treatment|R\/W'
    
    p = re.compile(r"(?i)("+order+r")?\s?("+step_names+r")")
    
    position_list = []
    
    for m in p.finditer(str_test):
        
        step = m.group()
        
        step = step.replace("draft","Draft").replace("polish","Polish")\
                             .replace('R/W','Rewrite').replace('rewrite','Rewrite')\
                             .replace("revision","Set of Revisions").replace("set of revisions","Set of Revisions")\
                             .replace("outline","Outline/Treatment").replace("treatment","Outline/Treatment")\
                            .replace("Outline","Outline/Treatment").replace("Treatment","Outline/Treatment")\
                            .replace("Outline/Outline/Treatment/Outline/Treatment","Outline/Treatment")\
                            .replace("first","1st").replace("second","2nd").replace("third",'3rd')\
                            .replace("First","1st").replace("Second","2nd").replace("Third",'3rd')\
                            .replace("fourth","4th").replace("fifth",'5th').replace('sixth','6th')\
                            .replace("Fourth","4th").replace("Fifth",'5th').replace('Sixth','6th')\
                            .lstrip(" ")
        
    
        catching_list.append(step)
        position_list.append(m.span())

    
    return (catching_list,position_list)

In [34]:
def writer_start_quali_condi_regex(str_test):
# For writer, start_quali_condi_regex is different, because there are only "On Commencement" and "On Delivery".

    catching_list = []
    
    p = re.compile(r"(?i)(comm|del)")
    
    position_list = []
    
    for m in p.finditer(str_test):
        catching_list.append(m.group().replace("comm","Commencement").replace("del","Delivery"))
        position_list.append(m.span())

    
    return (catching_list,position_list)

In [35]:
def writer_money_regex(str_test):
# For writer, start_quali_condi_regex is different, because there are only "On Commencement" and "On Delivery".

    catching_list = []
    
    p = re.compile(r"(?i)\$\W?([1-9]\d{0,2}[\,\.]?\d{,3}K?M?)")
    
    position_list = []
    
    for m in p.finditer(str_test):
        catching_list.append(m.group())
        position_list.append(m.span())

    
    return (catching_list,position_list)

In [36]:
def writer_comp_spliter(df, comp_id):
    
    
    rows = df[df["Right_COMPENSATION_ID"] == comp_id]
    indexs = rows.index
    
    for ind in indexs:
        print(comp_id)
        row = df.loc[ind]
        desc = row["split_desc"]
        
        
        # Catch Writing_step, money and condition
        
        writing_step_list, step_position_list = writing_step_regex(desc)
        quali_condi_list, quali_position_list = writer_start_quali_condi_regex(desc)
        #payment_type_list, payment_position_list = writer_payment_type_regex(desc)
        writer_money_list, money_position_list = writer_money_regex(desc)
        
        
        # only 1 writing step
        if len(writing_step_list) == 1:
            print("a")
            df.at[ind,"Compensation.Writing_Step"] = writing_step_list[0]
            df.at[ind,"Compensation.Writing_Step_Amount"] = df.loc[ind, 'Right_COMPENSATION_AMOUNT']
            continue
            
        #only 1 quali_condi
        if len(quali_condi_list) == 1 and len(writer_money_list) == 0:
            print("b")
            
            df.at[ind,"Compensation.Start_Qualifier"] = "On"
            df.at[ind,"Compensation.Payment_Start_Condition"] = quali_condi_list[0]
            continue
            
        # more than 1 writing step and no money
        if len(writing_step_list) > 1 and len(writer_money_list) == 0:
            print("c")
            amount = df.loc[ind, 'Right_COMPENSATION_AMOUNT']
            df = line_spliter(df, ind, len(writing_step_list))
            df.at[ind,"Compensation.Writing_Step"] = writing_step_list
            df.at[ind,"Compensation.Writing_Step_Amount"] = np.concatenate(([amount],
                                                                            np.repeat(0,len(writing_step_list) - 1)))
            continue
            


        # more than 1 pairs of writing step and money matched, no quali_condi
        if len(writing_step_list) > 1 and len(writer_money_list) > 1 and len(writing_step_list) == len(writer_money_list) \
        and len(quali_condi_list) == 0:
            print("d")
            df = line_spliter(df, ind, len(writing_step_list))
            df.at[ind,"Compensation.Writing_Step"] = writing_step_list
            
            transfered_money_list = []
            for money in writer_money_list:
                transfered_money_list.append(money_transfer(money))
            df.at[ind,"Compensation.Writing_Step_Amount"] = transfered_money_list
            continue
        
        # more than 1 pairs of quali_condi and money matched, no writing step.
        
        # Do I need to check if sum(writer_money_list) == 'Right_COMPENSATION_AMOUNT'?
        
                    #and sum(writer_money_list) == df.loc[ind, 'Right_COMPENSATION_AMOUNT']
        if len(quali_condi_list) > 1 and len(writer_money_list) > 1 and len(quali_condi_list) == len(writer_money_list) \
        and len(writing_step_list) == 0 :
            print("e")
            df = line_spliter(df, ind, len(quali_condi_list))
            transfered_money_list = []
            for money in writer_money_list:
                transfered_money_list.append(money_transfer(money))
            df.at[ind,"Compensation.%_Amount"] = transfered_money_list 
            df.at[ind,"Compensation.Start_Qualifier"] = np.repeat("On",len(quali_condi_list))
            df.at[ind,"Compensation.Payment_Start_Condition"] = quali_condi_list
            continue
            
        
        
        # there are all writing step, money and quali_condi, and they 3 may be not mathced.
        if len(quali_condi_list) > 1 and len(writer_money_list) > 1 and len(writing_step_list) >1 \
        and len(quali_condi_list) == len(writer_money_list) :
            print('f')
            df = line_spliter(df, ind, len(writer_money_list))
            transfered_money_list = []
            for money in writer_money_list:
                transfered_money_list.append(money_transfer(money))
                
            df.at[ind,"Compensation.%_Amount"] = transfered_money_list
            df.at[ind,"Compensation.Start_Qualifier"] = np.repeat("On",len(quali_condi_list))
            df.at[ind,"Compensation.Payment_Start_Condition"] = quali_condi_list    
            
            
            step_positions = [0]
            
            for step_position in step_position_list:
                step_positions.append(step_position[0])
                step_positions.append(step_position[1])
                
            step_position_list.append(len(desc))
            
            step_lists_repeated = []

            j = 0
            for i in range(0,len(writer_money_list)):
                if money_position_list[i][0] > step_position_list[2*j] and money_position_list[i][1]<step_position_list[2*j+1]:
                    step_lists_repeated.append(writing_step_list[j])
                elif i == 0:
                    continue
                else:
                    j = j + 2
           
            df.at[ind,"Compensation.Writing_Step"] = step_lists_repeated

    
    return df.reset_index(drop=True)

### Main Iteration Function

In [37]:
def split_pasting_func(func_df,func):
    df = func_df.copy()
    id_list = df.loc[df["Index"] == "New"]["Right_COMPENSATION_ID"].unique()
    for comp_id in id_list:
        print(func)
        print(comp_id)
        
        if func == "Financier" or func == "Generic_Functions" or func == "Researcher":
            continue
            
 
        if func == "Writer":
            
            df = overage_spliter(df, comp_id)

            df = Writer_Payment_Type_spliter(df, comp_id)
            
            # Core Part!!! About how to split Compensation_Amount---Writing_Step_Amount---%_Amount
            
            df = writer_comp_spliter(df, comp_id)
            # Core Part!!!
            
            
            df = dura_num_freq_paster(df, comp_id)
            df = rate_num_freq_paster(df, comp_id)
            df = start_date_paster(df, comp_id)
            df = fee_type_paster(df, func, comp_id)

            continue


        if func == "Rights":
            
            df = dura_num_freq_paster(df, comp_id)        
            df = Payment_Type_spliter(df, comp_id)         
            df = start_quali_condi_paster(df, comp_id)           
            df = total_guarant_paster(df, comp_id)
            df = check_number_paster(df, comp_id)
            df = start_expiry_date_paster(df, comp_id)
            df = date_paid_paster(df, comp_id)

            continue
        
        df = comp_type_paster(df, comp_id)
        #df = orig_deal_paster(df, comp_id)
        df = overage_spliter(df, comp_id)
        df = Payment_Type_spliter(df, comp_id)
        
        
        
        # How about combining these two functions together?
        df = per_amount_spliter(df, comp_id)
        df = start_quali_condi_paster(df, comp_id)
        
        
     
        df = dura_num_freq_paster(df, comp_id)
        df = rate_num_freq_paster(df, comp_id)
        df = start_date_paster(df, comp_id)
        
        
        df = fee_type_paster(df, func, comp_id)
        
        if func == "Producer":
            df = service_type_paster(df, func, comp_id)
       
            
        
    return df


In [38]:
def split_pasting_func_writer(func_df,func):
    df = func_df.copy()
    id_list = df.loc[df["Index"] == "New"]["Right_COMPENSATION_ID"].unique()
    for comp_id in id_list:
 
        if func == "Writer":
            print(func)
            print(comp_id)

            
            df = overage_spliter(df, comp_id)
        
            df = Writer_Payment_Type_spliter(df, comp_id)

            
            # Core Part!!! About how to split Compensation_Amount---Writing_Step_Amount---%_Amount
            
            df = writer_comp_spliter(df, comp_id)
            # Core Part!!!
            
            df = rate_num_freq_paster(df, comp_id)
            df = dura_num_freq_paster(df, comp_id)
            
            df = start_date_paster(df, comp_id)
            df = fee_type_paster(df, func, comp_id)

            continue
        
    return df


### Main Ioop

Using this dictionary to reorder the output columns.

In [39]:
column_order_list = {'Actor':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Compensation_Type","Compensation.Fee_Type",
                              "Compensation.Start_Date","Compensation.Duration_#","Compensation.Duration_Freq",
                              "Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Casting':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Compensation_Type","Compensation.Fee_Type",
                              "Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Consultant':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Compensation_Type","Compensation.Fee_Type",
                              "Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Director':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                                 "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",                                 
                              "Compensation.Compensation_Type","Compensation.Fee_Type","Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Service_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Financier':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Rights':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind","Compensation.Duration_#",
                               "Compensation.Duration_Freq","Compensation.Payment_Type","Compensation.%_Total",
                               "Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                               "Compensatipon.Total_Guaranteed_Commitment","Rights Start Condition",
                               "Rights Start Date","Compensation.Check_#","Compensation Commitment Date","# of Payments",
                               "Frequency","Start_Date","Expiry_Date","Purchase_Date","Reversion_Date",
                               "Date Paid","Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                    'Generic_Functions':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID",
                                         "COMPENSATION_AMOUNT","COMPENSATION_DESC","COMPENSATION_TYPE",
                                          "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                                          "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                                          "Right_PROJECT_ID"],
                     'Producer':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Compensation_Type","Compensation.Service_Type","Compensation.Fee_Type",
                              "Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"],
                     'Researcher':["DARTS_DIVISION","FUNCTION","COMPENSATION_ID","DEAL_ID","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Compensation_Type","Compensation.Service_Type","Compensation.Fee_Type",
                              "Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount",
                               "Compensation.Start_Qualifier",
                               "Index","Compensatipon.Total_Guaranteed_Commitment",
                              "Compensation.Service_Start_Condition","Right_DARTS_DIVISION",
                               "Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                               "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE",
                               "Right_DEAL_ID","Right_FUNCTION","Right_PROJECT_ID"],
                     'Writer':["DARTS_DIVISION","COMPENSATION_ID","DEAL_ID","FUNCTION","PROJECT_ID","COMPENSATION_AMOUNT",
                              "COMPENSATION_DESC","COMPENSATION_TYPE","Compensation.Applicable_ind",
                              "Compensation.Fee_Type",
                              "Compensation.Writing_Step","Compensation.Writing_Step_Amount","Compensation.Start_Date",
                              "Compensation.Duration_#","Compensation.Duration_Freq","Compensation.Rate",
                              "Compensation.Rate_Freq","Compensation.Payment_Type","Compensation.%_Total",
                              "Compensation.%_Amount","Compensation.Start_Qualifier","Compensation.Payment_Start_Condition",
                              "Index","Right_DARTS_DIVISION","Right_COMPENSATION_ID","Right_COMPENSATION_AMOUNT",
                              "Right_COMPENSATION_DESC","Right_COMPENSATION_TYPE","Right_DEAL_ID","Right_FUNCTION",
                              "Right_PROJECT_ID"]}

In [70]:
for func in dict_combined.keys():
    dict_combined[func] = split_pasting_func(dict_combined[func],func)
    #drop the temporary column
    dict_combined[func] = dict_combined[func].drop(columns = ["split_desc"])
    #One more step, choose excat columns with exact order.
    dict_combined[func] = dict_combined[func][column_order_list[func]]

Actor
5255
5255
5255
0
23
in the role of "Vivo", 
in the role of "Vivo", 
30
221
 per session with 5 guaranteed

1st opportunity to record dialogue and songs for Spanish-speaking territories at his election.  If he records Spanish-language dialogue in a separate session, 
 per session with 5 guaranteed

1st opportunity to record dialogue and songs for Spanish-speaking territories at his election.  If he records Spanish-language dialogue in a separate session, 
228
241
 per session.
 per session.
5255
in the role of "Vivo", $10,000 per session with 5 guaranteed

1st opportunity to record dialogue and songs for Spanish-speaking territories at his election.  If he records Spanish-language dialogue in a separate session, $10,000 per session.
in the role of "Vivo", $10,000 per session with 5 guaranteed

1st opportunity to record dialogue and songs for Spanish-speaking territories at his election.  If he records Spanish-language dialogue in a separate session, $10,000 per session.
5255
5255


1
5123
5123
 on commencement and 
 on commencement and 
5123
 on conclusion
 on conclusion
5123
5123
5123
5123
5123
5123
5123
5123
Director
5124
5124
5124
5124
per year directing salary; COS with 12-month guarantee at full-time start
                   START DATE:   tbd
per year directing salary; COS with 12-month guarantee at full-time start
                   START DATE:   tbd
5124
5124
5124
5124
Director
5125
5125
5125
5125
per year directing salary starting on commencement of production layout
per year directing salary starting on commencement of production layout
5125
5125
5125
5125
Director
5126
5126
5126
5126
per year directing salary starting 2 years after commencement of production layout through completion of services
per year directing salary starting 2 years after commencement of production layout through completion of services
5126
5126
5126
5126
Director
5136
5136
1
5136
5136
 on commencement and 
 on commencement and 
5136
 on comm of production, abandonment or end of 6 

1576805
1576805
 over 8 wks pre-prod; 
 over 8 wks pre-prod; 
1576805
 over sched prin photo; 
 over sched prin photo; 
1576805
 on compl dub/score; 
 on compl dub/score; 
1576805
 on compl del.
 on compl del.
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
1576805
Director
1577082
1577082
3
1577082
1577082
 over 8 wks pre-prod; 
 over 8 wks pre-prod; 
1577082
 over sched prin photo; 
 over sched prin photo; 
1577082
 on compl dub/score; 
 on compl dub/score; 
1577082
 on compl delivery.  $100K borne by John Davis.
 on compl delivery.  $100K borne by John Davis.
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
1577082
Financier
1576760
Financier
1576999
Rights
5204.0
5204.0
5204.0
5204.0
for an 18-month option (applicable) to acquire the rights in and to the project (including any written materials and drawings prepared in connection with his pi

0
182
Pilot:   Subject to close of license agreement and credit-based terms (eg sole written by or developed by credit), otherwise at producer's election, Aritst will be "locked" as EP at 
Pilot:   Subject to close of license agreement and credit-based terms (eg sole written by or developed by credit), otherwise at producer's election, Aritst will be "locked" as EP at 
189
361
 for exclusive services.  However if Artist is unavailable due to Disney obligations, then subject to credit-based terms, Artist will be a Consultant with episodic fees of 
 for exclusive services.  However if Artist is unavailable due to Disney obligations, then subject to credit-based terms, Artist will be a Consultant with episodic fees of 
368
369
.
.
5171
Pilot:   Subject to close of license agreement and credit-based terms (eg sole written by or developed by credit), otherwise at producer's election, Aritst will be "locked" as EP at $25,000 for exclusive services.  However if Artist is unavailable due to Di

signing bonus on execution of agreement
signing bonus on execution of agreement
5237
5237
5237
5237
5237
Producer
5239
5239
5239
5239
per year for producing services with 2 years guaranteed (may be pro-rated between "Fixed", "Black Knight" and HT4)  START DATE:  11/1/18
per year for producing services with 2 years guaranteed (may be pro-rated between "Fixed", "Black Knight" and HT4)  START DATE:  11/1/18
5239
5239
5239
5239
5239
Producer
5240
5240
5240
5240
per year, starting 2 years from start date (11/1/20) (may be pro-rated between "Fixed", "Black Knight" and HT4)
per year, starting 2 years from start date (11/1/20) (may be pro-rated between "Fixed", "Black Knight" and HT4)
5240
5240
5240
5240
5240
Producer
5241
5241
5241
5241
production bonus, payable on commencement of production layout
production bonus, payable on commencement of production layout
5241
5241
5241
5241
5241
Producer
5242
5242
5242
5242
production bonus, payable on complete delivery of the picture
production bonus, 

1576761
1576761
 on later of comm/exec & 
 on later of comm/exec & 
1576761
 on election to proceed/abandon, app.
 on election to proceed/abandon, app.
1576761
1576761
1576761
1576761
1576761
1576761
1576761
1576761
1576761
1576761
Producer
1576762
1576762
3
1576762
1576762
 over 8 wks pre-prod; 
 over 8 wks pre-prod; 
1576762
 over sched prin photo; 
 over sched prin photo; 
1576762
 on compl dub/score; 
 on compl dub/score; 
1576762
 on compl del, app vs contingent.
 on compl del, app vs contingent.
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
1576762
Producer
1576779
1576779
1576779
1576779
For min guaranty for 2 wks @ $13K/wk comm 2/11/19.  Pay DGA scale until signed.
For min guaranty for 2 wks @ $13K/wk comm 2/11/19.  Pay DGA scale until signed.
1576779
1576779
1576779
1576779
1576779
Producer
1576780
1576780
1576780
1576780
Per wk for EP svcs.
Per wk for EP svcs.
1576780
157

b
5293.0
5293.0
5293.0
5293.0
5293.0
5293.0
5293.0
5293.0
5293.0
Writer
5294.0
5294.0
5294.0
5294.0
5294.0
5294.0
5294.0
Writer
5295.0
5295.0
5295.0
5295.0
5295.0
5295.0
5295.0
Writer
5296.0
5296.0
5296.0
a
5296.0
5296.0
5296.0
5296.0
Writer
5297.0
5297.0
5297.0
c
1
5297.0
5297.0
5297.0
5297.0
5297.0
5297.0
5297.0
5297.0
Writer
5298.0
5298.0
5298.0
a
5298.0
5298.0
5298.0
5298.0
Writer
5299.0
5299.0
5299.0
a
5299.0
5299.0
5299.0
5299.0
Writer
5300.0
5300.0
5300.0
a
5300.0
5300.0
5300.0
5300.0
Writer
5311.0
5311.0
5311.0
5311.0
5311.0
5311.0
5311.0
Writer
5312.0
5312.0
5312.0
c
1
5312.0
5312.0
5312.0
5312.0
5312.0
5312.0
5312.0
5312.0
Writer
5313.0
5313.0
5313.0
a
5313.0
5313.0
5313.0
5313.0
Writer
5314.0
5314.0
5314.0
a
5314.0
5314.0
5314.0
5314.0
Writer
5317.0
5317.0
5317.0
c
1
5317.0
5317.0
5317.0
5317.0
5317.0
5317.0
5317.0
5317.0
Writer
5318.0
5318.0
5318.0
c
1
5318.0
5318.0
5318.0
5318.0
5318.0
5318.0
5318.0
5318.0
Writer
5319.0
5319.0
5319.0
a
5319.0
5319.0
5319.0
5319.0
Writer
53

### Excel Writer

In [71]:
#Writing excel
for func in dict_combined.keys():
    with pd.ExcelWriter(output_position + '/' + date + "_Fixed_" + func + "_" +
                              "Parsed.xlsx", engine='xlsxwriter') as writer:
        dict_combined[func].to_excel(writer, sheet_name='Fixed_Compensation',index = False)
        
print("Parsing complete.")

Parsing complete.


### Tester

In [46]:
test_df = dict_combined["Actor"]

In [53]:
test_df["Compensation.Start_Date"] = pd.to_datetime(test_df["Compensation.Start_Date"])

In [54]:
test_df.describe()

,Compensation.Applicable_ind,Compensation.Duration_#,Compensation.Rate,Compensation.%_Total,Compensation.%_Amount,COMPENSATION_ID,COMPENSATION_AMOUNT,DEAL_ID,Right_COMPENSATION_ID,Right_COMPENSATION_AMOUNT,Right_DEAL_ID
count,0.0,120.000000,143.000000,29.000000,27.000000,3.902000e+03,3.864000e+03,3902.00000,3.913000e+03,3.875000e+03,3913.000000
mean,NaN,4.708333,21780.136294,0.482414,109633.333333,1.129859e+06,1.084922e+06,18773.29959,1.128706e+06,1.083907e+06,18789.842065
std,NaN,3.452306,52182.440556,0.052689,199278.407559,5.451128e+05,3.002281e+06,13711.73171,5.463573e+05,2.998968e+06,13748.776516
min,NaN,1.000000,250.000000,0.330000,25.000000,8.700000e+01,0.000000e+00,5.00000,8.700000e+01,0.000000e+00,5.000000
25%,NaN,2.500000,10000.000000,0.500000,12500.000000,6.803510e+05,6.000000e+04,6803.50000,6.794020e+05,6.000000e+04,6794.000000
50%,NaN,5.000000,10000.000000,0.500000,25000.000000,1.521151e+06,1.250000e+05,15196.50000,1.521001e+06,1.250000e+05,15190.000000
75%,NaN,5.000000,10000.000000,0.500000,95000.000000,1.542468e+06,5.000000e+05,26573.50000,1.542546e+06,5.000000e+05,26580.000000
max,NaN,25.000000,500000.000000,0.500000,750000.000000,1.576700e+06,2.500000e+07,49435.00000,1.577079e+06,2.500000e+07,49680.000000


In [55]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3913 entries, 0 to 3912
Data columns (total 30 columns):
Compensation.Applicable_ind             0 non-null float64
Compensation.Compensation_Type          3696 non-null object
Compensation.Fee_Type                   3612 non-null object
Compensation.Start_Date                 94 non-null datetime64[ns]
Compensation.Duration_#                 120 non-null float64
Compensation.Duration_Freq              154 non-null object
Compensation.Rate                       143 non-null float64
Compensation.Rate_Freq                  142 non-null object
Compensation.Payment_Type               30 non-null object
Compensation.%_Total                    29 non-null float64
Compensation.%_Amount                   27 non-null float64
Compensation.Start_Qualifier            368 non-null object
Compensation.Payment_Start_Condition    417 non-null object
Index                                   3913 non-null object
DARTS_DIVISION                          390

In [108]:
float('0.5')

0.5

In [66]:
test_df = dict_combined["Generic_Functions"]

In [67]:
test_df.columns

Index(['DARTS_DIVISION', 'FUNCTION', 'COMPENSATION_ID', 'DEAL_ID',
       'PROJECT_ID', 'COMPENSATION_AMOUNT', 'COMPENSATION_DESC',
       'COMPENSATION_TYPE', 'Compensation.Applicable_ind',
       'Compensation.Compensation_Type', 'Compensation.Service_Type',
       'Compensation.Fee_Type', 'Compensation.Start_Date',
       'Compensation.Duration_#', 'Compensation.Duration_Freq',
       'Compensation.Rate', 'Compensation.Rate_Freq',
       'Compensation.Payment_Type', 'Compensation.%_Total',
       'Compensation.%_Amount', 'Compensation.Start_Qualifier', 'Index',
       'Compensatipon.Total_Guaranteed_Commitment',
       'Compensation.Service_Start_Condition', 'Right_DARTS_DIVISION',
       'Right_COMPENSATION_ID', 'Right_COMPENSATION_AMOUNT',
       'Right_COMPENSATION_DESC', 'Right_COMPENSATION_TYPE', 'Right_DEAL_ID',
       'Right_FUNCTION', 'Right_PROJECT_ID'],
      dtype='object')

In [64]:
test_df.columns

Index(['Compensation.Applicable_ind', 'Compensation.Compensation_Type',
       'Compensation.Service_Type', 'Compensation.Fee_Type',
       'Compensation.Start_Date', 'Compensation.Duration_#',
       'Compensation.Duration_Freq', 'Compensation.Rate',
       'Compensation.Rate_Freq', 'Compensation.Payment_Type',
       'Compensation.%_Total', 'Compensation.%_Amount',
       'Compensation.Start_Qualifier', 'Index', 'DARTS_DIVISION',
       'COMPENSATION_ID', 'COMPENSATION_AMOUNT', 'COMPENSATION_DESC',
       'COMPENSATION_TYPE', 'DEAL_ID', 'FUNCTION', 'PROJECT_ID',
       'Compensatipon.Total_Guaranteed_Commitment',
       'Compensation.Service_Start_Condition', 'Right_DARTS_DIVISION',
       'Right_COMPENSATION_ID', 'Right_COMPENSATION_AMOUNT',
       'Right_COMPENSATION_DESC', 'Right_COMPENSATION_TYPE', 'Right_DEAL_ID',
       'Right_FUNCTION', 'Right_PROJECT_ID'],
      dtype='object')

In [ ]:
test_rows = test_df.loc[20:23,:].reset_index(drop=True)

In [8]:
test_rows.Start_Date

0   NaT
1   NaT
2   NaT
3   NaT
Name: Start_Date, dtype: datetime64[ns]

In [35]:
test_rows = test_rows.append(test_rows.loc[2])

In [78]:
test_rows["Compensation.Rate"]

0        NaN
1    10000.0
2        NaN
3    10000.0
2        NaN
Name: Compensation.Rate, dtype: float64

In [149]:
'Compensation.Payment_Start_Condition' in test_df.columns

True

In [39]:
rows = test_rows[test_rows["Right_COMPENSATION_ID"] == "1647"]
indexs = rows.index

In [40]:
rows

,Compensation.Applicable_ind,Compensation.Compensation_Type,Compensation.Fee_Type,Compensation.Start_Date,Compensation.Duration_#,Compensation.Duration_Freq,Compensation.Rate,Compensation.Rate_Freq,Compensation.Payment_Type,Compensation.%_Total,...,PROJECT_ID,Right_DARTS_DIVISION,Right_COMPENSATION_ID,Right_COMPENSATION_AMOUNT,Right_COMPENSATION_DESC,Right_COMPENSATION_TYPE,Right_DEAL_ID,Right_FUNCTION,Right_PROJECT_ID,split_desc
2,NaN,Guaranteed,Acting Fee,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,KG0307,SPA,1647,100000.0,DBO bonuses tied to the initial theatrical rel...,Guaranteed,849.0,Talent,KG0307,DBO bonuses tied to the initial theatrical rel...
2,NaN,Guaranteed,Acting Fee,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,KG0307,SPA,1647,100000.0,DBO bonuses tied to the initial theatrical rel...,Guaranteed,849.0,Talent,KG0307,DBO bonuses tied to the initial theatrical rel...


In [41]:
indexs

Int64Index([2, 2], dtype='int64')

In [42]:
row_2 = test_rows[test_rows["Right_COMPENSATION_ID"] == "1646"]

In [43]:
index = row_2.index

In [44]:
row_2

,Compensation.Applicable_ind,Compensation.Compensation_Type,Compensation.Fee_Type,Compensation.Start_Date,Compensation.Duration_#,Compensation.Duration_Freq,Compensation.Rate,Compensation.Rate_Freq,Compensation.Payment_Type,Compensation.%_Total,...,PROJECT_ID,Right_DARTS_DIVISION,Right_COMPENSATION_ID,Right_COMPENSATION_AMOUNT,Right_COMPENSATION_DESC,Right_COMPENSATION_TYPE,Right_DEAL_ID,Right_FUNCTION,Right_PROJECT_ID,split_desc
1,NaN,Guaranteed,Acting Fee,NaT,5.0,Session,10000.0,Session,NaN,NaN,...,KG0307,SPA,1646,50000.0,"as ""Wayne"" (Werewolf), $10,000 per session wit...",Guaranteed,849.0,Talent,KG0307,"as ""Wayne"" (Werewolf), $10,000 per session wit..."


In [45]:
index

Int64Index([1], dtype='int64')

In [46]:
for i in index:
    print(i)

1
